In [ ]:
import easyocr
import pypdfium2 as pdfium
import cv2
import os
import re
from PIL import Image
import numpy as np

import matplotlib.pyplot as plt
import pdf2image

import torch

reader = easyocr.Reader(['en'])

In [ ]:
torch.cuda.is_available()

In [ ]:
import os

def list_pdf_files(directory):
    pdf_files = []
    for file in os.listdir(directory):
        if file.endswith(".pdf"):
            pdf_files.append(file)
    return pdf_files

# Example usage:
directory_path = os.getcwd()+"\\leases"
pdf_files = list_pdf_files(directory_path)
len(pdf_files)


In [ ]:
os.getcwd()

file = os.getcwd()+"\\leases\\0247_001.pdf"

poppler_path = r"D:\Release-23.08.0-0\poppler-23.08.0\Library\bin"

# images = pdfium.PdfDocument(file)

im = pdf2image.convert_from_path(file, poppler_path=poppler_path)

In [ ]:
im[0]
pic = im[0]
a = np.array(im[15])

b = a[100:1000] #page 1, landlord
c = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
c = cv2.bitwise_not(c)

test = Image.fromarray(c)
test

In [ ]:
results = reader.readtext(c, paragraph = True, batch_size = 4, detail = 0)
full_text = ' '.join([text for (text) in results])
print(full_text)

In [ ]:
#files_with_400 = [] #initialize list that will get all the file names with 400 forms inside thems

In [47]:
#main cell to count the number of 400 forms in the lease directory
z = 0
directory_path = os.getcwd()+"\\leases2\\"
for file in os.listdir(directory_path):
    if file.endswith(".pdf"):
        path = directory_path+file
        im = pdf2image.convert_from_path(path, poppler_path=poppler_path)

        for i, image in enumerate(im):
            a = np.array(image)
            b = a[100:1000] #page 1, landlord
            c = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
            c = cv2.bitwise_not(c)
            results = reader.readtext(c, paragraph = True, batch_size = 4, detail = 0)
            full_text = ' '.join([text for (text) in results])
            # print(full_text)
            if "Form 400" in full_text and "Schedule A" not in full_text:
                z += 1
                print(z, f"400 in file {file} in page {i}")
                files_with_400.append(file)
                break
            else:
                print(f"no 400 in file {file} in page {i}")
                pass

no 400 in file 0022_001.pdf in page 0
no 400 in file 0022_001.pdf in page 1
no 400 in file 0022_001.pdf in page 2
no 400 in file 0022_001.pdf in page 3
no 400 in file 0022_001.pdf in page 4
no 400 in file 0022_001.pdf in page 5
1 400 in file 0022_001.pdf in page 6
no 400 in file 0033_001.pdf in page 0
no 400 in file 0033_001.pdf in page 1
no 400 in file 0033_001.pdf in page 2
no 400 in file 0033_001.pdf in page 3
no 400 in file 0033_001.pdf in page 4
no 400 in file 0033_001.pdf in page 5
no 400 in file 0033_001.pdf in page 6
no 400 in file 0033_001.pdf in page 7
no 400 in file 0033_001.pdf in page 8
no 400 in file 0033_001.pdf in page 9
no 400 in file 0033_001.pdf in page 10
no 400 in file 0033_001.pdf in page 11
no 400 in file 0033_001.pdf in page 12
no 400 in file 0033_001.pdf in page 13
2 400 in file 0033_001.pdf in page 14
no 400 in file 0034_001.pdf in page 0
no 400 in file 0034_001.pdf in page 1
no 400 in file 0034_001.pdf in page 2
no 400 in file 0034_001.pdf in page 3
no 400 in

In [50]:
import pickle

with open('400_leases_list.pkl', 'wb') as f:
    pickle.dump(files_with_400, f)

# with open('400_leases_list.pkl', 'rb') as f:
#     files_with_400 = pickle.load(f)

In [48]:
len(files_with_400)

797

In [49]:
len(set(files_with_400))

797